<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_diffusers_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #Install dependencies
#@markdown You will need to restart runtime after that: `Runtime -> Restart Runtime`
!pip install --upgrade diffusers accelerate transformers xformers safetensors
!pip install pytorch-lightning omegaconf
!apt -y install -qq aria2
!wget https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml

!mkdir -p outputs/{txt2img,img2img}
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git StableDiffusionUi
!git clone https://huggingface.co/embed/negative /content/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/Lora/positive


# Creating model pipeline


<details>
<summary><b>Description</b><br>TLDR: use dropdown or just paste</summary>

---

You can just choose a model from a dropdown of popular models, which will also give you a link to a model page on [huggingface](https://huggingface.co) and any trigger words you need to use in the prompt.

Alternatively, you can paste a model_id, url, or local path for any model from huggingface:

* **model_id** looks like this: `gsdf/Counterfeit-V2.5`.
<br>You can use it for any *proper* diffusers model in huggingface, i.e. ones that have a lot of folders in their **Files**, and a file called **model_index.json**
<br> `A1111 format` toggle should be **<font color= 'red'>off</font>**.

* **url** may look like this:
```
https://huggingface.co/WarriorMama777/OrangeMixs/blob/main/Models/AbyssOrangeMix/AbyssOrangeMix.safetensors
```
Use it when the model is in Automatic1111 format, as in, just a single file.
<br> `A1111 format` toggle should be **<font color= 'green'>on</font>**.

* **local path** is just a path to a model folder (if diffusers format) or model file (if A1111 format).
<br>Set toggle accordingly.

Urls from civit.ai (the ones you get from clicking "Download") may also work, I'm not sure.

</details>

<details>
<summary>Where to find urls</summary>

A good place to find model urls is  [camenduru colabs repo](https://https://github.com/camenduru/stable-diffusion-webui-colab), just look for a line starting with `!aria2c ...`

</details>

In [ ]:
#@title Render model choice UI
output_index = 0
cache = None

from StableDiffusionUi.ColabUI.HugginfaceModelIndex import HugginfaceModelIndex

model_index = HugginfaceModelIndex("/content/StableDiffusionUi/model_index.json")
model_index.render()

In [ ]:
#@title Load chosen model
import torch
from diffusers import StableDiffusionPipeline

model_id, from_ckpt = model_index.get_model_id()
loader_func = StableDiffusionPipeline.from_single_file if from_ckpt else StableDiffusionPipeline.from_pretrained

pipe = loader_func(model_id,
                   custom_pipeline="lpw_stable_diffusion",
                   torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = None
pipe.enable_xformers_memory_efficient_attention()

<details>
<summary>Technical detail</summary>

I use [this](https://github.com/huggingface/diffusers/tree/main/examples/community#long-prompt-weighting-stable-diffusion) custom pipeline which doesn't have a token length limit and allows to use weights in prompt.

</details>

## Optional stuff:

In [ ]:
#@title Change sampler
from diffusers import EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, UniPCMultistepScheduler

choose_sampler = "UniPC" #@param ["Euler A", "DPM++", "DPM++ Karras", "UniPC"]

if choose_sampler == "Euler A":
    solver = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif choose_sampler == "DPM++":
    solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif choose_sampler == "DPM++ Karras":
    solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    solver.use_karras_sigmas = True
elif choose_sampler == "UniPC":
    solver = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.scheduler = solver
print(f"Sampler '{choose_sampler}' chosen")

### VAE

Most models come with their own VAE, but changing it may improve generation results. Especially if you get a lot of artifacts or just black images.

You can load VAE from any [huggingface](https://huggingface.co) repository, just paste their **model_id** and subfolder (usually just "vae").

In [ ]:
#@markdown ##(Optional) Load VAE

from diffusers import AutoencoderKL

vae_id_or_path = "waifu-diffusion/wd-1-5-beta2"  #@param {type: "string"}
vae_subfolder = "vae"    #@param {type: "string"}

vae = AutoencoderKL.from_pretrained(vae_id_or_path, subfolder=vae_subfolder, torch_dtype=torch.float16).to("cuda")
pipe.vae = vae

### Textual inversions

You can load any textual inversions using
```
pipe.load_textual_inversion("path/to/dir", weight_name="filename.pt")
```
To use them, you need to add theit token to a prompt like this `<filename>`.

Note that you need to reload textual inversions every time you switch VAE.

In [ ]:
#@markdown ###(Optional) Load textual inversions
#@markdown Load every embedding with **.pt** extension from **embeddings** folder.

import os

for path, subdirs, files in os.walk("/content/embeddings/"):
    for name in files:
        try:
            if os.path.splitext(name)[1] != ".pt": continue
            pipe.load_textual_inversion(path, weight_name=name)
            print(path, name)
        except: pass

### LoRA

Lora support is somewhat limited, and they make generation slower. I'm kinda lazy to write something that will make it more convenient, so for now we can just use code examples from [here](https://huggingface.co/docs/diffusers/main/en/training/lora#supporting-a1111-themed-lora-checkpoints-from-diffusers).

First, we can download loras, i.e. from Civitai like this (you can "Copy link" from download button):
```python
!wget https://civitai.com/api/download/models/15603 --content-disposition
```
To apply lora, run
```python
pipe.load_lora_weights("path/to/folder", weight_name="lora_filename.safetensors")
```
Then you can fuse lora, which will merge it's weights to model weights and make it faster. It also seem to be the only way to set lora scale:
```python
pipe.fuse_lora(lora_scale=0.5)
```
To unfuse lora (you need to use the same scale):
```python
pipe.unfuse_lora(lora_scale=0.5)
```

# Generating images

In [ ]:
#@title Render UI
#@markdown You don't need to run this cell again unless you want to change these settings
save_images = True #@param {type:"boolean"}
display_previewes = True    #@param {type:"boolean"}
#@markdown ---
from  StableDiffusionUi.ColabUI.DiffusionPipelineUI import DiffusionPipelineUI

ui = DiffusionPipelineUI()
if (cache is not None):
    ui.load_cache(cache)

ui.render()

In [ ]:
#@title <font color='red'>Run</font> this cell to generate images
cache = ui.get_dict_to_cache()

results = ui.generate(pipe)
if save_images:
    for i, image in enumerate(results.images):
        path = f"outputs/txt2img/{output_index:05}.png"
        ui.save_image_with_metadata(image, path, f"Batch: {i}\n")
        print(path)
        output_index += 1

if display_previewes:
    ui.display_image_previews(results.images)

#Utils

In [ ]:
#@markdown ## Image viewer
#@markdown Run this cell to view last results in full size
from IPython.display import clear_output
import ipywidgets as widgets

slider = widgets.IntSlider(max=len(results.images)-1)

def handler(change):
    slider.max = len(results.images)-1
    if change.new > slider.max: change.new = slider.max
    clear_output(wait=True)
    display(slider, results.images[change.new])

slider.observe(handler, names='value')

display(slider, results.images[slider.value])

## Converting models from ckpt
You shouldn't really need this, because diffusers now support loading from A1111 format.

But if you want, you can download a model and convert it to diffusers format.

NOTE: A good place to find model urls is  [camenduru colabs repo](https://https://github.com/camenduru/stable-diffusion-webui-colab), just look for a line starting with `!aria2c ...`


In [ ]:
#@markdown ##Download ckpt file
#@markdown Download ckpt/safetensors file from huggingface url.

#@markdown ---
import os

url = "https://huggingface.co/mekabu/MagicalMix_v2/resolve/main/MagicalMix_v2.safetensors" #@param {type:"string"}
ckpt_dump_folder = "/content/models_sd/"    #@param {type:"string"}
model_name = url.split('/')[-1]

bashCommand = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {ckpt_dump_folder} -o {model_name}"
os.system(bashCommand)

In [ ]:
#@markdown ##(Optional) Or just upload file
from google.colab import files
f = files.upload()

ckpt_dump_folder = "/content/"
for key in f.keys():
    model_name = key
    break

In [ ]:
#@markdown ##Run conversion script
#@markdown Paste a path to where you want this script to cache a model into `dump_path`.

#@markdown Keep `override_path` empty unless you uploaded your file to some custom directory.

import os
import torch
from diffusers.pipelines.stable_diffusion.convert_from_ckpt import download_from_original_stable_diffusion_ckpt

from_safetensors = True #@param {type:"boolean"}
override_path = "" #@param {type:"string"}
if override_path != "":
    checkpoint_path = override_path
else:
    checkpoint_path = os.path.join(ckpt_dump_folder, model_name)

pipe = download_from_original_stable_diffusion_ckpt(
        checkpoint_path=checkpoint_path,
        original_config_file = "/content/v1-inference.yaml",
        from_safetensors=from_safetensors,
    )

dump_path="models/ModelName/" #@param {type:"string"}
pipe.save_pretrained(dump_path, safe_serialization=from_safetensors)

pipe = pipe.to("cuda")
pipe.safety_checker = None
pipe.to(torch_dtype=torch.float16)

## Converting VAE

In [ ]:
#@markdown ##Download vae file
#@markdown Basically, the same thing as model files

#@markdown ---
import os

url = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt" #@param {type:"string"}
ckpt_dump_folder = "/content/models_sd/"    #@param {type:"string"}
model_name = url.split('/')[-1]

bashCommand = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {ckpt_dump_folder} -o {model_name}"
os.system(bashCommand)

In [ ]:
#@markdown Paste a path to where you want this script to cache a vae into `dump_path`.
from_safetensors = False #@param {type:"boolean"}
dump_path="vae/VaeName/" #@param {type:"string"}

checkpoint_path = os.path.join(ckpt_dump_folder, model_name)
bashCommand = f"python /content/StableDiffusionUi/scripts/convert_vae_pt_to_diffusers.py --vae_pt_path {checkpoint_path} --dump_path {dump_path}"
if from_safetensors:
    bashCommand += " --from_safetensors"
os.system(bashCommand)